In [19]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn import preprocessing
import numpy as np

In [22]:
df = pd.read_csv('AAPL.csv')
df = df.drop(['Date'], axis=1)

In [23]:
col = df.columns
x = df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled, columns=col)
df

,Open,High,Low,Close,Adj Close,Volume
0,0.009710,0.010496,0.009435,0.010456,0.009094,0.067250
1,0.010125,0.010268,0.009373,0.009305,0.008093,0.064077
2,0.009571,0.010260,0.009598,0.009488,0.008252,0.100102
3,0.009864,0.009826,0.008608,0.008389,0.007297,0.098700
4,0.008679,0.009095,0.008670,0.008939,0.007774,0.057082
...,...,...,...,...,...,...
5026,0.977842,0.969013,0.977132,0.967955,0.967968,0.001238
5027,0.978290,0.986384,0.983299,0.987223,0.987228,0.007285
5028,1.000000,1.000000,0.995149,0.986847,0.986853,0.014484
5029,0.994280,0.995632,0.985101,0.992723,0.992726,0.014193
